In [ ]:
!pip install selenium
!pip install BeautifulSoup4

     |████████████████████████████████| 911kB 13.9MB/s 


In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 86.0 MB of archives.
After this operation, 298 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 91.0.4472.101-0ubuntu0.18.04.1 [1,124 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 91.0.4472.101-0ubuntu0.18.04.1 [76.1 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 91.0.4472.101-0ubuntu0.1

In [ ]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains as ac
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

In [ ]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


아고다 호텔 리스트 저장된 csv파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
link_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트/한경_MLDL_호텔리뷰/rawdata/agoda_link_withpages.csv')
link_list.head()

,hotel_list,pages
0,https://www.agoda.com/ko-kr/josun-palace-a-lux...,1
1,https://www.agoda.com/ko-kr/shilla-stay-mapo/h...,53
2,https://www.agoda.com/ko-kr/nine-tree-hotel-do...,21
3,https://www.agoda.com/ko-kr/hotel-the-designer...,32
4,https://www.agoda.com/ko-kr/shilla-stay-gwangh...,69


크롬드라이버

In [ ]:
# 페이지 로딩 대기 5초
driver.implicitly_wait(5)

In [ ]:
len(link_list)

49

In [ ]:
links = []
pages = []
for i in range(0,len(link_list)):
    links.append(link_list.iloc[i]['hotel_list'])
    pages.append(link_list.iloc[i]['pages'])
len(links), len(pages)

(49, 49)

In [ ]:
links[2]

'https://www.agoda.com/ko-kr/nine-tree-hotel-dongdaemun/hotel/seoul-kr.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1891463&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2021-07-9&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tag=45b17d1d-e0b0-fe2a-ce90-5513829d856b&tspTypes=1&los=1&searchrequestid=ddd2399d-638e-4364-8cdc-b2645f8cee4b'

In [ ]:
review_sum_list = []
for link in links:
    driver.get(link)
    time.sleep(5)
    # 총 리뷰 개수 찾기: for문을 몇 번 돌릴지 구하기위함 (버튼을 몇 번 눌러야 할지) 
    review_sum = driver.find_element_by_css_selector("#reviewFilterSection > div.Review__SummaryContainer > span").text
    time.sleep(3)
    try : 
        review_sum = review_sum.split(' ')[3][:-1]
    except:
        review_sum = review_sum.split(' ')[1][:-1] 
    review_sum_list.append(review_sum)

print(review_sum_list)

KeyboardInterrupt: ignored

In [ ]:
review_sum_list

['모',
 '1043',
 '417',
 '629',
 '1372',
 '2851',
 '750',
 '648',
 '1163',
 '339',
 '521',
 '633',
 '1611',
 '5580',
 '832',
 '848',
 '449',
 '2802',
 '1000',
 '8511',
 '2104',
 '1568',
 '241',
 '114',
 '870',
 '1192',
 '140',
 '652',
 '1125',
 '138',
 '336',
 '761',
 '527',
 '1026',
 '181',
 '408',
 '460',
 '721',
 '2483',
 '606',
 '1918',
 '514',
 '1012',
 '411',
 '89',
 '1068',
 '777',
 '784',
 '226']

In [ ]:
review_sum_list[0] = '1'
review_sum_list[:5]

['1', '1043', '417', '629', '1372']

In [ ]:
pages = []

for num in review_sum_list:
    page = int(num)//20 + 1
    pages.append(page)

print(pages)

[1, 53, 21, 32, 69, 143, 38, 33, 59, 17, 27, 32, 81, 280, 42, 43, 23, 141, 51, 426, 106, 79, 13, 6, 44, 60, 8, 33, 57, 7, 17, 39, 27, 52, 10, 21, 24, 37, 125, 31, 96, 26, 51, 21, 5, 54, 39, 40, 12]


In [ ]:
link_list['pages'] = pages

In [ ]:
link_list.head()

,hotel_list,pages
0,https://www.agoda.com/ko-kr/josun-palace-a-lux...,1
1,https://www.agoda.com/ko-kr/shilla-stay-mapo/h...,53
2,https://www.agoda.com/ko-kr/nine-tree-hotel-do...,21
3,https://www.agoda.com/ko-kr/hotel-the-designer...,32
4,https://www.agoda.com/ko-kr/shilla-stay-gwangh...,69


In [ ]:
link_list.to_csv('data/agoda_link_withpages.csv',encoding='utf-8-sig',index=False)

In [ ]:
from google.colab import files

In [ ]:
texts = []
hotels = []

for i in range(12,50):
    driver.get(links[i])
    time.sleep(1)
    for v in range(1, pages[i]+1) :
        
        for j in range(0,20):
            try : 
                text = driver.find_element_by_css_selector(f'#review-{j} > div.Review-comment-right > div.Review-comment-bubble > div.Review-comment-body > p.Review-comment-bodyText').text
                time.sleep(1)
                texts.append(text)
                hotel = driver.find_element_by_css_selector('#property-critical-root > div > div.Northstar > div.sc-bdfBwQ.cXTIvA > div.NorthstarMainContent > div:nth-child(2) > div.HeaderCerebrum > div:nth-child(1) > h1').text
                time.sleep(1)
                hotels.append(hotel)
            except:
                pass
        print(f'{i}번째 호텔, {v}번째 페이지')
        time.sleep(2)
        if pages[i] != 1 :
            #driver.find_element_by_css_selector('#reviewSection > div:nth-child(6) > div > span:nth-child(3) > i').send_keys(Keys.ENTER)
            try :
              element = driver.find_element_by_css_selector('#reviewSection > div:nth-child(5) > div > span:nth-child(3) > i')
              driver.execute_script("arguments[0].click();", element)
            except : 
              element = driver.find_element_by_css_selector('#reviewSection > div:nth-child(6) > div > span:nth-child(3) > i')
              driver.execute_script("arguments[0].click();", element)
    

    hotel_r = pd.DataFrame({'호텔 이름' : hotels, '리뷰': texts})
    hotel_r.to_csv(f'agoda_review_4부터_{i}까지 완료.csv', index=False, encoding='utf-8-sig')

12번째 호텔, 1번째 페이지
12번째 호텔, 2번째 페이지
12번째 호텔, 3번째 페이지
12번째 호텔, 4번째 페이지
12번째 호텔, 5번째 페이지
12번째 호텔, 6번째 페이지
12번째 호텔, 7번째 페이지
12번째 호텔, 8번째 페이지
12번째 호텔, 9번째 페이지
12번째 호텔, 10번째 페이지
12번째 호텔, 11번째 페이지
12번째 호텔, 12번째 페이지
12번째 호텔, 13번째 페이지
12번째 호텔, 14번째 페이지
12번째 호텔, 15번째 페이지
12번째 호텔, 16번째 페이지
12번째 호텔, 17번째 페이지
12번째 호텔, 18번째 페이지
12번째 호텔, 19번째 페이지
12번째 호텔, 20번째 페이지
12번째 호텔, 21번째 페이지
12번째 호텔, 22번째 페이지
12번째 호텔, 23번째 페이지
12번째 호텔, 24번째 페이지
12번째 호텔, 25번째 페이지
12번째 호텔, 26번째 페이지
12번째 호텔, 27번째 페이지
12번째 호텔, 28번째 페이지
12번째 호텔, 29번째 페이지
12번째 호텔, 30번째 페이지
12번째 호텔, 31번째 페이지
12번째 호텔, 32번째 페이지
12번째 호텔, 33번째 페이지
12번째 호텔, 34번째 페이지
12번째 호텔, 35번째 페이지
12번째 호텔, 36번째 페이지
12번째 호텔, 37번째 페이지
12번째 호텔, 38번째 페이지
12번째 호텔, 39번째 페이지
12번째 호텔, 40번째 페이지
12번째 호텔, 41번째 페이지
12번째 호텔, 42번째 페이지
12번째 호텔, 43번째 페이지
12번째 호텔, 44번째 페이지
12번째 호텔, 45번째 페이지
12번째 호텔, 46번째 페이지
12번째 호텔, 47번째 페이지
12번째 호텔, 48번째 페이지
12번째 호텔, 49번째 페이지
12번째 호텔, 50번째 페이지
12번째 호텔, 51번째 페이지
12번째 호텔, 52번째 페이지
12번째 호텔, 53번째 페이지
12번째 호텔, 54번째 페이지
12번째 호텔, 55번째 페이지
12번째 호텔, 56번째 페이지
1

MaxRetryError: ignored

In [ ]:
len(texts)

121

In [ ]:
links[13
]

'https://www.agoda.com/ko-kr/nine-tree-hotel-myeong-dong/hotel/seoul-kr.html?finalPriceView=1&isShowMobileAppPrice=false&cid=1891463&numberOfBedrooms=&familyMode=false&adults=2&children=0&rooms=1&maxRooms=0&checkIn=2021-07-9&isCalendarCallout=false&childAges=&numberOfGuest=0&missingChildAges=false&travellerType=1&showReviewSubmissionEntry=false&currencyCode=KRW&isFreeOccSearch=false&tag=45b17d1d-e0b0-fe2a-ce90-5513829d856b&tspTypes=3&los=1&searchrequestid=ddd2399d-638e-4364-8cdc-b2645f8cee4b'

In [ ]:
#reviewSection > div:nth-child(6) > div > span:nth-child(3) > i